In [ ]:
import os
import imageio
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.backends.cudnn as cudnn
import torchvision.utils as vutils
cudnn.benchmark = True

import pytorch3d
import pytorch3d.io


# some utility function for visualization
import utils
from utils.util_3d import init_mesh_renderer, sdf_to_mesh, add_mesh_textures

# some utility function
from utils.qual_util import save_mesh_as_gif
from IPython.display import Image as ipy_image
from IPython.display import display

%reload_ext autoreload
%autoreload 2

In [ ]:
from utils.demo_util import get_shape_comp_opt
from utils.demo_util import get_shape_comp_model

""" setup dir """
res_dir = 'results'
if not os.path.exists(res_dir): os.mkdir(res_dir)

""" setup opt"""
gpu_id = 0

opt = get_shape_comp_opt(gpu_id=gpu_id)

""" setup different model """
model = get_shape_comp_model(opt)    
model.eval()

""" setup renderer """
dist, elev, azim = 1.7, 20, 110
mesh_renderer = init_mesh_renderer(image_size=256, dist=dist, elev=elev, azim=azim, device=opt.device)

# Shape Completion
1. Unconditional generation
2. Structured input (e.g., given a leg of a chair)

In [ ]:
""" Shape completion 1 - unconditional generation """
gen_sdf = model.uncond_gen(bs=9, topk=31)
gen_sdf_as_mesh = sdf_to_mesh(gen_sdf)

# save as gif
gif_name = f'{res_dir}/shape-comp-uncond.gif'
save_mesh_as_gif(mesh_renderer, gen_sdf_as_mesh, nrow=3, out_name=gif_name)

ipy_image(gif_name)

In [ ]:
"""
    Shape completion 2 - structured input
     - given a mesh, we first extract SDF from that mesh.
     - then user can specify the partial input by setting the min & max values of x, y, z.
"""
import utils
from utils.qual_util import get_partial_shape_by_range
from preprocess.process_one_mesh import process_obj

# input: complete chair
# first extract sdf from mesh
obj_file = 'demo_data/chair_model.obj'
sdf_file = process_obj(obj_file)

sdf = utils.util_3d.read_sdf(sdf_file).to(opt.device)


In [ ]:
from utils.qual_util import get_partial_shape_by_range
#sdf = model.uncond_gen(bs=1, topk=31)
sdf = torch.load('./demo_data/voxelChair.pt',map_location=torch.device('cuda'))
gen_sdf_as_mesh = sdf_to_mesh(sdf)
print (sdf.shape)
# save as gif
gif_name = f'{res_dir}/shape-example.gif'
save_mesh_as_gif(mesh_renderer, gen_sdf_as_mesh, nrow=1, out_name=gif_name)
ipy_image(gif_name)


In [ ]:
#print(sdf)
torch.save(sdf, 'demo_data/tensor.txt')

In [ ]:

# range: -1 ~ 1.
# x: left-to-right; y: bottom-to-top; z: front-to-back
# example: only conditioning on the bottom of the chair
min_x, max_x = -1., 1.
min_y, max_y = 0., 1.
min_z, max_z = -1., 1.
input_range = {'x1': min_x, 'x2': max_x, 'y1': min_y, 'y2': max_y, 'z1': min_z, 'z2': max_z}

shape_comp_input = get_partial_shape_by_range(sdf, input_range)

In [ ]:
""" perform shape completion """
input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=9, topk=30)

gen_mesh = sdf_to_mesh(comp_sdf)          # completed shape

# save as gif
gif_name = f'{res_dir}/shape-comp-struct-input-inp.gif'
save_mesh_as_gif(mesh_renderer, input_mesh, nrow=1, out_name=gif_name)

gen_gif_name = f'{res_dir}/shape-comp-struct-input-gen.gif'
save_mesh_as_gif(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name)

for name in [gif_name, gen_gif_name]:
    display(ipy_image(name))